# Creating datatypes

In [1]:
from recordclass import dataobject

## LinkedList

In [10]:
class LinkedItem(dataobject):
    val: object
    next: "LinkedItem"

class LinkedList(dataobject):
    start: LinkedItem
    end: LinkedItem
        
    def append(self, val):
        link = LinkedItem(val, None)
        if self.start is None:
            self.start = link
            self.end = link
        else:
            self.end.next = link
            self.end = link
            
#     def extend(self, vals):
#         iter_vals = iter(vals)

#         val = next(iter_vals)
#         if self.start is None:
#             self.start = link
#             self.end = link
#         else:
#             self.end.next = link
#             self.end = link

#         for val in iter_vals:
#             self.end.next = link
#             self.end = link
    
#     def remove(self, val):
#         start = self.start
#         if start is None:
#             return
        
#         end = self.end
        
#         curr = start
#         if curr.val == val:
#             self.start = curr.next
#             if self.end is curr:
#                 self.end = None
#             curr.next = None
#             return

#         prev = curr
#         curr = curr.next

#         while curr.next is not None:
#             if curr.val == val:
#                 prev.next = curr.next
#                 return
                    
#             prev = curr
#             curr = curr.next


In [11]:
ll = LinkedList(None, None)

In [12]:
ll.append(1)

In [14]:
ll.start, ll.end

(LinkedItem(val=1, next=None), LinkedItem(val=1, next=None))

In [15]:
ll.append(2)

In [16]:
ll

LinkedList(start=LinkedItem(val=1, next=LinkedItem(val=2, next=None)), end=LinkedItem(val=2, next=None))